In [1]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [4]:
df=pd.read_csv("C:/Users/maths/Desktop/FML-AVI-230957164/Womens Clothing E-Commerce Reviews.csv")
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [5]:
print(df.isnull().sum())
df=df.dropna()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64


In [6]:
review_count=df['Clothing ID'].value_counts()
df_filtered=df[df['Clothing ID'].isin(review_count[review_count>5].index)]
df=df_filtered.sort_values(by='Clothing ID',ascending=True)
df.head(20)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
16698,16698,8,38,Cute suit-didn't work for me,"This suit was high quality, and a cute design....",1,0,0,Initmates,Intimate,Swim
16695,16695,8,33,Great coverage,This bathing suit has everything. i especially...,5,1,2,Initmates,Intimate,Swim
16731,16731,8,33,Love this suit!,"I am 5'5, 150lb, generally size medium shirt, ...",5,1,0,Initmates,Intimate,Swim
16711,16711,8,70,Alas not as good as it looks,"The suit is lovely, but it is very long in the...",2,0,0,Initmates,Intimate,Swim
16737,16737,8,36,Not what i expected,I bought this suit based on the great reviews....,2,0,0,Initmates,Intimate,Swim
16703,16703,8,57,Retro style,I bought a size 8 a few years back not my regu...,5,1,0,Initmates,Intimate,Swim
16739,16739,8,42,Such a nice bathing suit!,"This is my first one piece bathing suit, and i...",5,1,0,Initmates,Intimate,Swim
18528,18528,24,37,Love this swimsuit!,This swimsuit is super cute and retro. the ruc...,5,1,0,Initmates,Intimate,Swim
18523,18523,24,39,Worth it!,This suit is great for curvy and slightly flaw...,5,1,0,Initmates,Intimate,Swim
18535,18535,24,29,Super unflattering,"I'm 5' 5"", 150 lbs, 32dd, normally a 28p or 29...",2,0,0,Initmates,Intimate,Swim


In [7]:
contractions_dict = {
    "don't": "do not", "won't": "will not", "can't": "cannot", "i'm": "i am", 
    "it's": "it is", "that's": "that is", "he's": "he is", "she's": "she is", 
    "we're": "we are", "they're": "they are", "you're": "you are", 
    "didn't": "did not", "doesn't": "does not", "wasn't": "was not", "weren't": "were not",
    "hasn't": "has not", "haven't": "have not", "i've": "i have", "we've": "we have",
    "you've": "you have", "they've": "they have", "i'll": "i will", "you'll": "you will",
    "he'll": "he will", "she'll": "she will", "we'll": "we will", "they'll": "they will",
    "i'd": "i would", "you'd": "you would", "he'd": "he would", "she'd": "she would",
    "we'd": "we would", "they'd": "they would"
}
def expand_contractions(text):
    for contraction, expansion in contractions_dict.items():
        text = text.replace(contraction, expansion)
    return text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = expand_contractions(text)  # Expand contractions
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text
df['Review Text'] = df['Review Text'].apply(clean_text)
df['Title'] = df['Title'].apply(clean_text)
df.head(20)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
16698,16698,8,38,cute suitdid not work for me,this suit was high quality and a cute design h...,1,0,0,Initmates,Intimate,Swim
16695,16695,8,33,great coverage,this bathing suit has everything i especially ...,5,1,2,Initmates,Intimate,Swim
16731,16731,8,33,love this suit,i am 55 150lb generally size medium shirt size...,5,1,0,Initmates,Intimate,Swim
16711,16711,8,70,alas not as good as it looks,the suit is lovely but it is very long in the ...,2,0,0,Initmates,Intimate,Swim
16737,16737,8,36,not what i expected,i bought this suit based on the great reviews ...,2,0,0,Initmates,Intimate,Swim
16703,16703,8,57,retro style,i bought a size 8 a few years back not my regu...,5,1,0,Initmates,Intimate,Swim
16739,16739,8,42,such a nice bathing suit,this is my first one piece bathing suit and i ...,5,1,0,Initmates,Intimate,Swim
18528,18528,24,37,love this swimsuit,this swimsuit is super cute and retro the ruch...,5,1,0,Initmates,Intimate,Swim
18523,18523,24,39,worth it,this suit is great for curvy and slightly flaw...,5,1,0,Initmates,Intimate,Swim
18535,18535,24,29,super unflattering,i am 5 5 150 lbs 32dd normally a 28p or 29p in...,2,0,0,Initmates,Intimate,Swim


In [8]:
x=df['Review Text']
y=df['Recommended IND']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

vectorizer = CountVectorizer(stop_words='english')
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

# Apply Naïve Bayes classifier
model = MultinomialNB()
model.fit(x_train_vectorized, y_train)

# Step 5: Make predictions on the test set
y_pred = model.predict(x_test_vectorized)

# Evaluate the model using precision, recall, F1 score, and accuracy
precision = precision_score(y_test, y_pred, average='binary', pos_label=1)  # Replace with actual positive label
recall = recall_score(y_test, y_pred, average='binary', pos_label=1)  # Replace with actual positive label
f1 = f1_score(y_test, y_pred, average='binary', pos_label=1)  # Replace with actual positive label
accuracy = accuracy_score(y_test, y_pred)

# Output the evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')



Accuracy: 0.8926
Precision: 0.9126
Recall: 0.9612
F1 Score: 0.9363


In [9]:
user_review=[" disgusting "]
user_review_vec=vectorizer.transform(user_review)
predicted=model.predict(user_review_vec)
if predicted==1:
    print("the product is recommended🥰")
    
else:
    print("this product is shit!!! 💩")

this product is shit!!! 💩
